## Install dependencies

In [23]:
# !pip install openpyxl
# !pip install pandas

## Set up environment and creds

In [24]:
import os
import calendar
import pandas as pd
import logging as log
from google.cloud import bigquery as bq
from google.oauth2 import service_account

JSON_KEYS_PATH = '../json-keys/gch-prod-dwh01-data-pipeline.json'

# set up credentials for BQ and Drive to query data
credentials = service_account.Credentials.from_service_account_file(JSON_KEYS_PATH)
bq_client = bq.Client(credentials=credentials, project=credentials.project_id)

## Read expected results into a dataframe

In [25]:
expected_df = pd.read_excel('srp.xlsx', sheet_name='Sheet3', header=0)
all_dates = expected_df['date'].unique()

## Check if all unique dates are extracted correctly

In [26]:
all_dates

<DatetimeArray>
['2024-08-01 00:00:00', '2024-08-02 00:00:00', '2024-08-03 00:00:00',
 '2024-08-04 00:00:00', '2024-08-05 00:00:00', '2024-08-06 00:00:00',
 '2024-09-01 00:00:00', '2024-09-02 00:00:00', '2024-09-03 00:00:00',
 '2024-09-04 00:00:00', '2024-09-05 00:00:00', '2024-09-06 00:00:00',
 '2024-09-07 00:00:00', '2024-09-08 00:00:00', '2024-09-09 00:00:00',
 '2024-09-10 00:00:00', '2024-09-11 00:00:00', '2024-09-12 00:00:00',
 '2024-09-13 00:00:00', '2024-09-14 00:00:00', '2024-09-15 00:00:00',
 '2024-09-16 00:00:00', '2024-09-17 00:00:00', '2024-09-18 00:00:00',
 '2024-09-19 00:00:00', '2024-09-20 00:00:00', '2024-10-01 00:00:00',
 '2024-10-02 00:00:00', '2024-10-03 00:00:00', '2024-10-04 00:00:00',
 '2024-10-05 00:00:00', '2024-10-06 00:00:00', '2024-10-07 00:00:00',
 '2024-10-08 00:00:00', '2024-10-09 00:00:00', '2024-10-10 00:00:00',
 '2024-10-11 00:00:00', '2024-10-12 00:00:00', '2024-10-13 00:00:00',
 '2024-10-14 00:00:00', '2024-10-15 00:00:00']
Length: 41, dtype: datetime

In [27]:
len(all_dates)

41

## Inspect expected_results df

In [28]:
expected_df.head()

,date,total_qty_sales,total_sales,total_margin
0,2024-08-01,21236.2142,126738.61,29163.94
1,2024-08-02,18902.6783,125851.20,29148.37
2,2024-08-03,26079.3416,169499.20,36124.83
3,2024-08-04,26414.1901,183690.14,39238.73
4,2024-08-05,20538.1360,129752.49,30793.10


### Rename ```total_sales_qty``` to ```total_qty_sales```

Somehow the name for ```total_qty_sales``` is not registered properly even after modification

In [29]:
expected_df = expected_df.rename(columns={
		'total_sales_qty': 'total_qty_sales'
})

## Inspect expected_df again

In [30]:
expected_df.head()

,date,total_qty_sales,total_sales,total_margin
0,2024-08-01,21236.2142,126738.61,29163.94
1,2024-08-02,18902.6783,125851.20,29148.37
2,2024-08-03,26079.3416,169499.20,36124.83
3,2024-08-04,26414.1901,183690.14,39238.73
4,2024-08-05,20538.1360,129752.49,30793.10


## Get all mismatched data

Join all mismatches into a dataframe

In [31]:
mismatch_df = pd.DataFrame()

for date in all_dates:
	# Convert to date string in the format BigQuery expects
	formatted_date = date.strftime('%Y-%m-%d')
	
	query = f"""
	SELECT
		date,
		SUM(total_qty_sales) AS total_qty_sales,
		SUM(total_sales) AS total_sales,
		SUM(total_margin) AS total_margin
	FROM `gch-prod-dwh01.srp.agg_possales_copy5`
	WHERE date = DATE '{formatted_date}'
	GROUP BY date
	"""

	results_df = bq_client.query(query).to_dataframe()
	mismatch_df = pd.concat([mismatch_df, results_df], ignore_index=True, sort=False)


## Inspect the mismatched data

In [32]:
mismatch_df.head()

,date,total_qty_sales,total_sales,total_margin
0,2024-08-01,11721.2201,44670.89,9635.88
1,2024-08-02,9555.8705,43142.50,10286.87
2,2024-08-03,13903.5728,56188.88,11392.43
3,2024-08-04,13970.0486,64507.47,13068.66
4,2024-08-05,11566.8014,43778.06,10385.05


## Create a comparison table

Set ```different``` as ```True``` if there are any mismatches

In [ ]:
comparison_df = pd.DataFrame({
	'date': mismatch_df['date'],
	'expected_total_qty_sales': round(expected_df['total_qty_sales'], 2),
	'cur_total_qty_sales': round(mismatch_df['total_qty_sales'], 2),
	'expected_sales': round(expected_df['total_sales'], 2),
	'cur_sales': round(mismatch_df['total_sales'], 2),
	'expected_margin': round(expected_df['total_margin'], 2),
	'cur_margin': round(mismatch_df['total_margin'], 2)
})

comparison_df['different'] = (
	(comparison_df['expected_total_qty_sales'] != comparison_df['cur_total_qty_sales']) | 
	(comparison_df['expected_sales'] != comparison_df['cur_sales']) | 
	(comparison_df['expected_margin'] != comparison_df['cur_margin'])
)

final = comparison_df[comparison_df['different'] == True]
final

In [35]:
mismatch_df.memory_usage(deep=True).sum()

np.int64(1444)

In [36]:
(2.4 * 1024 ** 3) / 1444

1784612.4498614958

In [55]:
def show_null_data(location):
    print(f"=== NULL DATA (Location: {location}) ===")
    for date in expected_df['date'].unique():
        formatted_date = date.strftime('%Y-%m-%d')
        query = f"""
        SELECT 
            date,
            SUM(total_qty_sales) AS total_qty_sales
        FROM `gch-prod-dwh01.srp_data.srp_possales_{location}_copy2` 
        WHERE date = '{formatted_date}'
        GROUP BY date
        """
        results_df = bq_client.query(query).to_dataframe()
        if not results_df.empty:
            print(f"Date: {date}")
            display(results_df)


In [56]:
show_null_data(1216)

=== NULL DATA (Location: 1216) ===


In [ ]:
show_null_data(1217)

In [ ]:
locations = [1216, 1217, 1221, 1226, 1227, 1228]
date_loc_missing = []

for date in expected_df['date'].unique():
	formatted_date = date.strftime('%Y-%m-%d')
	
	for loc in locations:
		query = f"""
		SELECT date, total_qty_sales
		FROM `gch-prod-dwh01.srp_data.srp_possales_{loc}_copy2` 
		WHERE date = '{formatted_date}'
		"""
		
		try:
			results_df = bq_client.query(query).to_dataframe()
			
			if results_df.empty:
				date_loc_missing.append((formatted_date, loc))
				
		except Exception as e:
			print(f"Error querying location {loc} for date {formatted_date}: {str(e)}")
			date_loc_missing.append((formatted_date, loc))  # Assume missing if error occurs

print(date_loc_missing)

[('2024-08-01', 1216), ('2024-08-01', 1221), ('2024-08-01', 1227), ('2024-08-01', 1228), ('2024-08-02', 1216), ('2024-08-02', 1221), ('2024-08-02', 1227), ('2024-08-02', 1228), ('2024-08-03', 1216), ('2024-08-03', 1221), ('2024-08-03', 1227), ('2024-08-03', 1228), ('2024-08-04', 1216), ('2024-08-04', 1221), ('2024-08-04', 1227), ('2024-08-04', 1228), ('2024-08-05', 1216), ('2024-08-05', 1221), ('2024-08-05', 1227), ('2024-08-05', 1228), ('2024-08-06', 1216), ('2024-08-06', 1221), ('2024-08-06', 1227), ('2024-08-06', 1228), ('2024-09-01', 1216), ('2024-09-01', 1227), ('2024-09-01', 1228), ('2024-09-02', 1216), ('2024-09-02', 1227), ('2024-09-02', 1228), ('2024-09-03', 1216), ('2024-09-03', 1227), ('2024-09-03', 1228), ('2024-09-04', 1216), ('2024-09-04', 1227), ('2024-09-04', 1228), ('2024-09-05', 1216), ('2024-09-05', 1227), ('2024-09-05', 1228), ('2024-09-06', 1216), ('2024-09-06', 1227), ('2024-09-06', 1228), ('2024-09-07', 1216), ('2024-09-07', 1227), ('2024-09-07', 1228), ('2024-09

In [ ]:
def extract_and_push_missing_data(date_loc_missing):
    creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT, scopes=SCOPES)
    drive_service = build('drive', 'v3', credentials=creds)
    
    for date_str, loc in date_loc_missing:
        try:
            # Convert date to folder structure
            date_obj = datetime.strptime(date_str, '%Y-%m-%d')
            folder_name = f"SRP_{date_obj.strftime('%Y%m')}"
            file_name = f"SRP_{loc}_{date_obj.strftime('%Y%m%d')}.csv"
            
            log.info(f"Processing {file_name} in folder {folder_name}")
            
            # 1. Find the parent folder
            folder_query = f"name = '{folder_name}' and '{SRP_PARENT_FOLDER_ID}' in parents and mimeType = 'application/vnd.google-apps.folder'"
            folder = drive_service.files().list(
                q=folder_query,
                fields='files(id)'
            ).execute().get('files', [])
            
            if not folder:
                log.error(f"Folder {folder_name} not found")
                continue
                
            folder_id = folder[0]['id']
            
            # 2. Find the CSV file in the folder
            file_query = f"name = '{file_name}' and '{folder_id}' in parents"
            csv_file = drive_service.files().list(
                q=file_query,
                fields='files(id, name)'
            ).execute().get('files', [])
            
            if not csv_file:
                log.error(f"File {file_name} not found in {folder_name}")
                continue
                
            # 3. Process and upload the CSV
            results_df = process_csv_from_drive(drive_service, csv_file[0])
            
            if not results_df.empty:
                # Add proper date formatting
                results_df['date'] = pd.to_datetime(results_df['date']).dt.date
                
                # Standardize column names
                results_df.columns = [snake_case(col) for col in results_df.columns]
                
                # Push to BigQuery
                load_table(results_df, f"{loc}_copy2")
                log.info(f"Successfully uploaded {file_name} to location {loc}")
            else:
                log.warning(f"Empty dataframe for {file_name}")
                
        except Exception as e:
            log.error(f"Failed to process {file_name}: {str(e)}")
            continue

# Modified load_table function with proper partitioning
def load_table(df, table_suffix: str):
    table_ref = f'gch-prod-dwh01.srp_data.srp_possales_{table_suffix}'
    job_config = bq.LoadJobConfig(
        write_disposition='WRITE_APPEND',
        time_partitioning=bq.TimePartitioning(field="date"),
        autodetect=True
    )
    job = bq_client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    return job.result()

In [65]:
a = '2024-08-01'

a = a.replace('-', '')[:6]
# a = a[:6]

print(a)

202408


In [1]:
l = [('2024-08-01', 1216), ('2024-08-01', 1221), ('2024-08-01', 1227), ('2024-08-01', 1228), ('2024-08-02', 1216), ('2024-08-02', 1221), ('2024-08-02', 1227), ('2024-08-02', 1228), ('2024-08-03', 1216), ('2024-08-03', 1221), ('2024-08-03', 1227), ('2024-08-03', 1228), ('2024-08-04', 1216), ('2024-08-04', 1221), ('2024-08-04', 1227), ('2024-08-04', 1228), ('2024-08-05', 1216), ('2024-08-05', 1221), ('2024-08-05', 1227), ('2024-08-05', 1228), ('2024-08-06', 1216), ('2024-08-06', 1221), ('2024-08-06', 1227), ('2024-08-06', 1228), ('2024-09-01', 1216), ('2024-09-01', 1227), ('2024-09-01', 1228), ('2024-09-02', 1216), ('2024-09-02', 1227), ('2024-09-02', 1228), ('2024-09-03', 1216), ('2024-09-03', 1227), ('2024-09-03', 1228), ('2024-09-04', 1216), ('2024-09-04', 1227), ('2024-09-04', 1228), ('2024-09-05', 1216), ('2024-09-05', 1227), ('2024-09-05', 1228), ('2024-09-06', 1216), ('2024-09-06', 1227), ('2024-09-06', 1228), ('2024-09-07', 1216), ('2024-09-07', 1227), ('2024-09-07', 1228), ('2024-09-08', 1216), ('2024-09-08', 1227), ('2024-09-08', 1228), ('2024-09-09', 1216), ('2024-09-09', 1227), ('2024-09-09', 1228), ('2024-09-10', 1216), ('2024-09-10', 1227), ('2024-09-10', 1228), ('2024-09-11', 1216), ('2024-09-11', 1227), ('2024-09-11', 1228), ('2024-09-12', 1216), ('2024-09-12', 1227), ('2024-09-12', 1228), ('2024-09-13', 1216), ('2024-09-13', 1227), ('2024-09-13', 1228), ('2024-09-14', 1216), ('2024-09-14', 1227), ('2024-09-14', 1228), ('2024-09-15', 1216), ('2024-09-15', 1227), ('2024-09-15', 1228), ('2024-09-16', 1216), ('2024-09-16', 1227), ('2024-09-16', 1228), ('2024-09-17', 1216), ('2024-09-17', 1227), ('2024-09-17', 1228), ('2024-09-18', 1216), ('2024-09-18', 1227), ('2024-09-18', 1228), ('2024-09-19', 1216), ('2024-09-19', 1227), ('2024-09-19', 1228), ('2024-09-20', 1216), ('2024-09-20', 1227), ('2024-09-20', 1228), ('2024-10-01', 1216), ('2024-10-01', 1217), ('2024-10-01', 1227), ('2024-10-01', 1228), ('2024-10-02', 1216), ('2024-10-02', 1217), ('2024-10-02', 1227), ('2024-10-02', 1228), ('2024-10-03', 1216), ('2024-10-03', 1217), ('2024-10-03', 1227), ('2024-10-04', 1216), ('2024-10-04', 1227), ('2024-10-05', 1216), ('2024-10-05', 1227), ('2024-10-06', 1216), ('2024-10-06', 1227), ('2024-10-07', 1216), ('2024-10-07', 1217), ('2024-10-07', 1227), ('2024-10-08', 1216), ('2024-10-08', 1217), ('2024-10-08', 1227), ('2024-10-09', 1216), ('2024-10-09', 1217), ('2024-10-09', 1227), ('2024-10-10', 1216), ('2024-10-10', 1217), ('2024-10-10', 1227), ('2024-10-11', 1216), ('2024-10-11', 1217), ('2024-10-11', 1227), ('2024-10-12', 1216), ('2024-10-12', 1217), ('2024-10-12', 1227), ('2024-10-13', 1216), ('2024-10-13', 1217), ('2024-10-13', 1227), ('2024-10-14', 1216), ('2024-10-14', 1217), ('2024-10-15', 1216), ('2024-10-15', 1217), ('2024-10-15', 1227)]

print(len(l))

127
